In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import time
import psycopg2
import numpy as np
import datetime

In [2]:
link = 'https://lk.ranepa.ru/pk/list.php?FT=1&FL=0&FK=&FC=&FB=&FO=&F1=149cf41f-6a2b-11e8-80f0-000c299441de&F2=66fef4e2-c613-11e5-80e4-005056a00b6e&F3=6e9ce087-c78b-4932-88a5-2628115de4b3'

In [3]:
def find_href(link):
    try:
        req = requests.get(link)
        soup = BeautifulSoup(req.text)
        table = soup.find('table')
        href = []
        t = table.find_all('a', href=True)
        for item in t:
            href.append('https://lk.ranepa.ru/pk/list.php'+item['href'])
    except:
        href = []
    return href



def make_table(href):
    try:
        reqq = requests.get(href)
        soup = BeautifulSoup(reqq.text)
        name_of_program = soup.find('div', {'class':'wrapper abitur-list'}).find('p').find_all('a')[-1].text

        data = pd.read_html(href)
        free = data[0]
        free['type_of_cost'] = 'free'
        free['program'] = name_of_program
        payed = data[1]
        payed['type_of_cost'] = 'payed'
        payed['program'] = name_of_program
    except:
        payed = pd.DataFrame()
        free = pd.DataFrame()
    
    return pd.concat([payed, free])


def get_data(link):
    df = pd.DataFrame()
    list_of_href = find_href(link)
    for i in range(len(list_of_href)):
        inner_table = make_table(list_of_href[i])
        df = pd.concat([inner_table, df], join='outer')
        time.sleep(3)
    return df




In [4]:
def create_table():
    command = """CREATE TABLE ranepa(
    № INT,
    ФИО VARCHAR(255),
    Номер_дела VARCHAR(255),
    Сумма_баллов_по_индивидуальным_достижениям INT,
    Русский_язык INT,
    Математика INT,
    Информатика_иИКТ INT,
    Обществознание INT,
    type_of_cost VARCHAR(255),
    program VARCHAR(255),
    Иностранный_язык INT)"""
    
    try:
        conn = psycopg2.connect(user="postgres",
                                          
                                          password="",
                                          host="localhost",
                                          port="5432",
                                          database="parsing")
        cur = conn.cursor()

        cur.execute(command)
    
        cur.close()
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [5]:
##updates table

from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

def bulkInsert(records):
    try:
        conn = psycopg2.connect(user="postgres",
                                          
                                          password="",
                                          host="localhost",
                                          port="5432",
                                          database="parsing")
        cursor = conn.cursor()
        sql_insert_query = """ INSERT INTO ranepa(№,ФИО,
                                                     Номер_дела,
                                                     Сумма_баллов_по_индивидуальным_достижениям,
                                                     Русский_язык,
                                                     Математика,
                                                     Информатика_иИКТ,
                                                     Обществознание,
                                                     type_of_cost,
                                                     program,
                                                     Иностранный_язык) 
                           VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) """

        # executemany() to insert multiple rows
        result = cursor.executemany(sql_insert_query, records)
        conn.commit()
        print(cursor.rowcount, "Record inserted successfully into mobile table")

    except (Exception, psycopg2.Error) as error:
        print("Failed inserting record into table {}".format(error))

    finally:
        # closing database connection.
        if conn:
            cursor.close()
            conn.close()
            print("PostgreSQL connection is closed")



In [25]:
datt.columns = ['№',
 'ФИО',
 'Номер_дела',
 'Сумма_баллов_по_индивидуальным_достижениям',
 'Русский_язык',
 'Математика',
 'Информатика_иИКТ',
 'Обществознание',
 'type_of_cost',
 'program',
 'Иностранный_язык']
datt.fillna(0, inplace=True)
datt = datt.replace('-', 0)
datt[['№','Информатика_иИКТ', 'Обществознание', 'Иностранный_язык', 'Математика', 'Сумма_баллов_по_индивидуальным_достижениям', 'Русский_язык']] = datt[['№', 'Информатика_иИКТ', 'Обществознание', 'Иностранный_язык','Математика', 'Сумма_баллов_по_индивидуальным_достижениям', 'Русский_язык']].astype('int64')

In [6]:
create_table()
data = get_data(link)

data.columns = ['№',
 'ФИО',
 'Номер_дела',
 'Сумма_баллов_по_индивидуальным_достижениям',
 'Русский_язык',
 'Математика',
 'Информатика_иИКТ',
 'Обществознание',
 'type_of_cost',
 'program',
 'Иностранный_язык']
data.fillna(0, inplace=True)
data = data.replace('-', 0)


data[['№','Информатика_иИКТ', 'Обществознание', 'Иностранный_язык', 'Математика', 'Сумма_баллов_по_индивидуальным_достижениям', 'Русский_язык']] = data[['№', 'Информатика_иИКТ', 'Обществознание', 'Иностранный_язык','Математика', 'Сумма_баллов_по_индивидуальным_достижениям', 'Русский_язык']].astype('int64')


records = list(data.to_records(index=False))
bulkInsert(records)



478 Record inserted successfully into mobile table
PostgreSQL connection is closed


In [8]:
while True:
    num_rows = len(data)
    
    data = get_data(link)
    
    data.columns = ['№',
     'ФИО',
     'Номер_дела',
     'Сумма_баллов_по_индивидуальным_достижениям',
     'Русский_язык',
     'Математика',
     'Информатика_иИКТ',
     'Обществознание',
     'type_of_cost',
     'program',
     'Иностранный_язык']
    data.fillna(0, inplace=True)
    data = data.replace('-', 0)


    data[['№','Информатика_иИКТ', 'Обществознание', 'Иностранный_язык', 'Математика', 'Сумма_баллов_по_индивидуальным_достижениям', 'Русский_язык']] = data[['№', 'Информатика_иИКТ', 'Обществознание', 'Иностранный_язык','Математика', 'Сумма_баллов_по_индивидуальным_достижениям', 'Русский_язык']].astype('int64')
    
    added_data = data.iloc[num_rows:, :]
    
    records = list(added_data.to_records(index=False))
    bulkInsert(records)
    print(datetime.datetime.now())
    time.sleep(100)
    

0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:05:17.933643
0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:07:25.293443
0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:09:28.545192
0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:11:34.701275
0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:13:38.082792
0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:15:42.220938
0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:17:51.154166
0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:19:54.935426
0 Record inserted successfully into mobile table
PostgreSQL connection is closed
2021-07-02 15:22:01.822271
0 Record inserted successful

KeyboardInterrupt: 